# Experiment 07: Final

 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *
from modules.evaluation import *

from modules.logging import *
from modules.train import *

import time

CPU = False
if CPU:
    # Set CPU as available physical device
    tf.config.set_visible_devices([], 'GPU')
else:
    # Check GPUs and enable dynamic GPU memory use:",
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                # Prevent TensorFlow from allocating all memory of all GPUs:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e07-8', 'e07-4']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[1]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRE_BUILD = True          # Step 1 of the training process
PRETRAIN = False          # Step 1 of the training process
GAN_BUILD = True          # Step 1 of the training process
GAN_TRAIN = False         # Step 2 of the training process
BICUBIC_EVALUATE_LAST = False
PRE_EVALUATE_LAST = False
GAN_EVALUATE_LAST = False
PRE_EVALUATE_HISTORY = False
GAN_EVALUATE_HISTORY = False
HR_EVALUATE = True  # Compute Ma and NIQE for PAN instead of SR!

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e07'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

### CONVERT WV02 VAL IMAGES TO TRAIN!!! ##########################
def val_to_train(meta):
    meta.loc[(meta['train_val_test'] == 'val') 
             & (meta['sensorVehicle'] == 'WV02') , 'train_val_test'] = 'train'
    return meta
meta = val_to_train(meta)
##################################################################

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 10.0,  ################################# <--- The oversampling of patches
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.127841.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
    NEW_PIXEL_SIZE_PAN = 0.5
    RESIZE_DIR = DATA_PATH + '-resized'
    
# Data augmentation
AUGMENT_FLIP = True # both up/down and left/right flips
AUGMENT_ROTATE = True # 90 degree rotations
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e07-8': {'train': 'WV02', 'test': ['WV02']}, 
               'e07-6': {'train': 'WV02', 'test': ['WV02']}, 
               'e07-4': {'train': 'WV02', 'val': ['GE01'], 'test': ['WV02', 'GE01']}, 
               'e07-3': {'train': 'WV02', 'val': ['GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e07-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e07-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e07-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e07-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e07-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                         'e07-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                   if k not in ['Coastal', 'NIR2']}, 
                         'e07-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                   if k in ['Blue', 'Green', 'Red', 'NIR']},
                         'e07-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e07-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e07-8': {None: None},                                   # not enough bands in GE01
                         'e07-6': {None: None},                                    # not enough bands in GE01
                         'e07-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                         'e07-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e07-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 16 # Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 0     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 1
VAL_N_BATCHES_SAVE = 1

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 0
# Proportion of val batches that will go through ma and niqe metric calculation
# MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
MA_NIQE_PROPORTION = 1  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_DIRS = {'e07-4': LOGS_EXP_DIR + '/models/' + 'e07-4-pre_20210401-212113/'
                        }
PRETRAIN_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIRS[EXPERIMENT]
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_DIR + EXPERIMENT + '-pre-400.h5'

# Path to the gan-training weights that will be 
GAN_WEIGHTS_DIRS = {'e07-4': LOGS_EXP_DIR + '/models/' + 'e07-4-gan_20210404-102643/'
                   }
GAN_WEIGHTS_DIR = GAN_WEIGHTS_DIRS[EXPERIMENT]
GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-399.h5'
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = True
METRIC_NIQE = True
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
#################################################################################################################

### EVALUTAION ##################################################################################################
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    METRIC_MA = True
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        

EVAL_STEPS_PER_EPOCH = 'all'
EVAL_N_EPOCHS = 400
#EVAL_SENSOR = 'WV02'
EVAL_PER_IMAGE = True
    
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        
#if PRE_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 1
#    EVAL_PREFIX = EXPERIMENT + '-pre-'
#elif GAN_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = GAN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 0
#    EVAL_PREFIX = EXPERIMENT + '-gan-'
    
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 32, 'val': 9, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4, 'NIR': 6}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2, 'NIR': 3}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: True
NIQE: True
Perceptual Index (PI): True


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [3]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 935412, 'val': 3709, 'test': 9278}
Total number of tiles: 948399


### 2.3 Tile generation to disk

In [4]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [5]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [6]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 346.7 and sd 134.9 from json file @ data/toulon-laspezia-tiles/e07/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [7]:
SHUFFLE_BUFFER_SIZE = {'train': 90000,  # 100
                       'val': 8000,  # 100
                       'test': 8000}  # 100

#SHUFFLE_BUFFER_SIZE = {'train': 100,  # 100
#                       'val': 100,  # 100
#                       'test': 100}  # 100
#
train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-' 
                                                  + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test],
                                   augment_flip=True,
                                   augment_rotate=True
                                  )
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.2044782512000547
Output (float) min, max: -0.20389012705396095 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [8]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'test' #### <- Special only for e07
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [9]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [10]:
if PRE_BUILD:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [11]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [12]:
if GAN_BUILD:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH, 
                                   metric_ma_niqe_hr=HR_EVALUATE)

Starting matlab.engine ...
matlab.engine started


## 7. GAN training

In [13]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

## Predict single

## 8. Evaluation

### 8.1 Data input pipelines for final evaluation

The pipeline is modified to include the file paths of the tiles/patches so that it is possible to log performance metrics for individual files and by extension for individual satellite images.

#### 8.1.2 Test set

In [14]:
train_val_test = 'test'
ds_test = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_test[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                     batch_size=BATCH_SIZE[train_val_test], 
                                     ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                     pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                     sensor=sensor,
                                     band_selection=band_indices, 
                                     mean_correction=train_tiles_mean,
                                     cache_memory=False,
                                     cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_filepath_cache'), 
                                     repeat=False, 
                                     shuffle=False, 
                                     shuffle_buffer_size=0, 
                                     include_file_paths=True)
    ds_test[sensor] = ds_test[sensor].get_dataset()
print(ds_test.keys())

dict_keys(['WV02', 'GE01'])


### 8.2 Evaluate last epoch

In [15]:
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST or HR_EVALUATE:
    val_or_test = 'test'

    if PRE_EVALUATE_LAST and not GAN_EVALUATE_LAST:
        PRE_GAN = ['pre']
    if not PRE_EVALUATE_LAST and GAN_EVALUATE_LAST:
        PRE_GAN = ['gan']
    if HR_EVALUATE:
        PRE_GAN = ['HR']

    print(PRE_GAN, SENSORS[val_or_test])
    for pre_gan in PRE_GAN:
        
        for sensor in SENSORS[val_or_test]:
            # if sensor == 'GE01':
            #     continue
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
            elif pre_gan == 'gan':
                gan_model.G.load_weights(GAN_WEIGHTS_PATH)
            elif pre_gan == 'HR':
                # load pretrain weights. The model does not use the weights during HR evaluation
                gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
            else:
                raise NotImplementedError

            print(pre_gan, sensor)
            start = time.time()
            results_df = esrgan_evaluate(model=gan_model, 
                                         dataset=ds_test[sensor], 
                                         steps='all', 
                                         per_image=True, 
                                         write_csv=True,
                                         csv_path=str(LOGS_EXP_DIR + '/csv/' + 'final_epoch-' 
                                                      + pre_gan + '-' + val_or_test + '-' + sensor + '.csv'), 
                                         verbose=1
                                        )
            end = time.time()
            print(str((end - start) / 60), 'minutes')

['HR'] ['WV02', 'GE01']
HR WV02
Computed 8 images in  139.73062562942505 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 34.67627716064453, 'SSIM': 0.855657696723938, 'Ma': 4.521763801574707, 'NIQE': 4.436053276062012, 'PI': 4.957144737243652}
Computed 8 images in  130.43807530403137 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 47.23466110229492, 'SSIM': 0.9965898394584656, 'Ma': 3.8637051582336426, 'NIQE': 7.9408674240112305, 'PI': 7.038580894470215}
Computed 8 images in  127.42508029937744 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 35.35295486450195, 'SSIM': 0.873033881187439, 'Ma': 4.38671875, 'NIQE': 4.071873664855957, 'PI': 4.8425774574279785}
Computed 8 images in  124.13161158561707 seconds
L

Computed 8 images in  124.71895694732666 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 38.85805892944336, 'SSIM': 0.978363037109375, 'Ma': 4.485186576843262, 'NIQE': 5.813120365142822, 'PI': 5.663967132568359}
Computed 8 images in  125.8187906742096 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 36.97526550292969, 'SSIM': 0.8727247714996338, 'Ma': 4.310225486755371, 'NIQE': 4.769021987915039, 'PI': 5.229398250579834}
Computed 8 images in  124.98851466178894 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 54.55551528930664, 'SSIM': 0.9986308217048645, 'Ma': 2.9371018409729004, 'NIQE': 8.589409828186035, 'PI': 7.826153755187988}
Computed 8 images in  124.3474690914154 seconds
Last image: {'G_pixel_loss': 

Computed 8 images in  124.78720450401306 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 31.92158317565918, 'SSIM': 0.8169909715652466, 'Ma': 4.601229190826416, 'NIQE': 4.3794426918029785, 'PI': 4.889106750488281}
Computed 8 images in  124.91435408592224 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 39.487022399902344, 'SSIM': 0.9201520681381226, 'Ma': 4.067597389221191, 'NIQE': 4.5065202713012695, 'PI': 5.219461441040039}
Computed 8 images in  124.82575845718384 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 35.10481643676758, 'SSIM': 0.8636645674705505, 'Ma': 4.57008695602417, 'NIQE': 4.164149284362793, 'PI': 4.797031402587891}
Computed 8 images in  124.63424873352051 seconds
Last image: {'G_pixel_los

Computed 8 images in  121.42365837097168 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 37.55410385131836, 'SSIM': 0.9557332396507263, 'Ma': 4.555304050445557, 'NIQE': 4.833118915557861, 'PI': 5.138907432556152}
Computed 8 images in  120.50675988197327 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 43.4005012512207, 'SSIM': 0.9610962271690369, 'Ma': 4.200094223022461, 'NIQE': 5.096802234649658, 'PI': 5.4483537673950195}
Computed 8 images in  120.58886075019836 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 45.150665283203125, 'SSIM': 0.9799000024795532, 'Ma': 4.6893086433410645, 'NIQE': 5.721380233764648, 'PI': 5.516036033630371}
Computed 8 images in  120.49616694450378 seconds
Last image: {'G_pixel_los

Computed 8 images in  120.89938592910767 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 42.938785552978516, 'SSIM': 0.9559889435768127, 'Ma': 4.223759174346924, 'NIQE': 9.033186912536621, 'PI': 7.4047136306762695}
Computed 8 images in  121.16107082366943 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 41.09810256958008, 'SSIM': 0.9576696753501892, 'Ma': 4.54510498046875, 'NIQE': 5.050107002258301, 'PI': 5.252501010894775}
Computed 8 images in  120.69855165481567 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 39.47085952758789, 'SSIM': 0.8957895040512085, 'Ma': 4.135571479797363, 'NIQE': 6.085966110229492, 'PI': 5.9751973152160645}
Computed 8 images in  123.01667594909668 seconds
Last image: {'G_pixel_los

...

Computed 8 images in  119.96449208259583 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 19.982669830322266, 'SSIM': 0.7959685325622559, 'Ma': 4.704555988311768, 'NIQE': 4.539778232574463, 'PI': 4.917611122131348}
Computed 8 images in  120.41580200195312 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 29.07807731628418, 'SSIM': 0.9035243988037109, 'Ma': 4.691479206085205, 'NIQE': 4.157961368560791, 'PI': 4.733241081237793}
Computed 8 images in  120.48150730133057 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 22.196041107177734, 'SSIM': 0.4463197886943817, 'Ma': 4.616748332977295, 'NIQE': 3.9823546409606934, 'PI': 4.682803153991699}
Computed 8 images in  120.5929102897644 seconds
Last image: {'G_pixel_los

Computed 8 images in  121.6538577079773 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 36.60009002685547, 'SSIM': 0.8603123426437378, 'Ma': 4.818936347961426, 'NIQE': 3.739656925201416, 'PI': 4.460360527038574}
Computed 8 images in  123.32952666282654 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 47.594547271728516, 'SSIM': 0.9949391484260559, 'Ma': 2.8909239768981934, 'NIQE': 5.466944694519043, 'PI': 6.288010597229004}
Computed 8 images in  121.75805068016052 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 39.72063446044922, 'SSIM': 0.9041599035263062, 'Ma': 4.948886394500732, 'NIQE': 4.424142360687256, 'PI': 4.737627983093262}
Computed 8 images in  121.2976655960083 seconds
Last image: {'G_pixel_loss'

Computed 8 images in  120.05082058906555 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 34.45865249633789, 'SSIM': 0.8639683723449707, 'Ma': 4.92706823348999, 'NIQE': 3.908294677734375, 'PI': 4.490612983703613}
Computed 8 images in  120.54638719558716 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 35.196044921875, 'SSIM': 0.8196901679039001, 'Ma': 4.487425327301025, 'NIQE': 3.9229671955108643, 'PI': 4.717771053314209}
Computed 8 images in  120.20912957191467 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 33.46559524536133, 'SSIM': 0.7741210460662842, 'Ma': 4.706946849822998, 'NIQE': 3.682739019393921, 'PI': 4.487895965576172}
Computed 8 images in  120.50173997879028 seconds
Last image: {'G_pixel_loss': 

Computed 8 images in  121.19160676002502 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 47.09251022338867, 'SSIM': 0.9959325790405273, 'Ma': 3.0189225673675537, 'NIQE': 9.057339668273926, 'PI': 8.019208908081055}
Computed 8 images in  121.74234533309937 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 34.33748245239258, 'SSIM': 0.7955687642097473, 'Ma': 4.704768180847168, 'NIQE': 3.7727842330932617, 'PI': 4.534008026123047}
Computed 8 images in  120.79987597465515 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 36.96348190307617, 'SSIM': 0.8912533521652222, 'Ma': 4.277103424072266, 'NIQE': 4.169152736663818, 'PI': 4.9460248947143555}
Computed 8 images in  122.25963687896729 seconds
Last image: {'G_pixel_lo

Computed 8 images in  121.53987526893616 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 32.048919677734375, 'SSIM': 0.8257415294647217, 'Ma': 4.683722019195557, 'NIQE': 3.8557546138763428, 'PI': 4.5860161781311035}
Computed 8 images in  120.82923889160156 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 27.665264129638672, 'SSIM': 0.7258263826370239, 'Ma': 5.003928184509277, 'NIQE': 4.822067737579346, 'PI': 4.909070014953613}
Computed 8 images in  120.4423565864563 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 32.266326904296875, 'SSIM': 0.7931299805641174, 'Ma': 5.170100688934326, 'NIQE': 4.769706726074219, 'PI': 4.799802780151367}
Computed 8 images in  120.61955380439758 seconds
Last image: {'G_pixel_l

### 8.3 Evaluate every kth epoch

In [ ]:
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    val_or_test = 'test'
    
    # Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
    if METRIC_MA:
        PRE_GAN = ['gan']
    else:
        PRE_GAN = ['pre', 'gan']

    for pre_gan in PRE_GAN:
        for sensor in SENSORS[val_or_test]:
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                model_weights_dir = PRETRAIN_WEIGHTS_DIR
                eval_first_step = 1
                eval_prefix = EXPERIMENT + '-pre-'
            else:
                model_weights_dir = GAN_WEIGHTS_DIR
                eval_first_step = 0
                eval_prefix = EXPERIMENT + '-gan-'

            esrgan_epoch_evaluator(gan_model,
                                   model_weights_dir=model_weights_dir,
                                   model_weight_prefix=eval_prefix,
                                   dataset=ds_test[sensor],
                                   n_epochs=EVAL_N_EPOCHS,
                                   first_epoch=eval_first_step,
                                   steps_per_epoch=EVAL_STEPS_PER_EPOCH,
                                   k_epoch=25,
                                   csv_dir=LOGS_EXP_DIR + '/csv/' + val_or_test + '-' + sensor, 
                                       per_image=EVAL_PER_IMAGE, 
                                   verbose=0)

### 8.4 Comparison plots

In [ ]:
def list_all_ms_tiles(tiles_dir):
    if isinstance(tiles_dir, str):
        tiles_dir = pathlib.Path(tiles_dir)
    ms_tiles_paths = [p for p in tiles_dir.glob('**/ms/*.tif')]
    return ms_tiles_paths

def esrgan_predict_multiple(
    ms_tiles_paths, 
    results_dir, 
    esrgan_generator_model,
    bicubic_model,
    pretrain_weights_path, 
    gan_weights_path,
    wv02_band_indices,
    ge01_band_indices,
    mean_correction,
    sr_factor, 
    geotiff_or_png='png'):
    if isinstance(results_dir, str):
        results_dir = pathlib.Path(results_dir)
    results_dir.mkdir(parents=True, exist_ok=True)
    
    if geotiff_or_png == 'png':
        output_dtype = 'uint8'
    elif geotiff_or_png == 'geotiff':
        output_dtype = 'uint16' 
    else:
        raise ValueError
    
    count = 0
    for ms_tile_path in ms_tiles_paths:
        results_img_dir = results_dir.joinpath(ms_tile_path.parents[1].stem)
        results_img_dir.mkdir(parents=True, exist_ok=True)
        if 'WV02' in str(ms_tile_path):
            sensor = 'WV02'
            band_indices = wv02_band_indices
        elif 'GE01' in str(ms_tile_path):
            sensor = 'GE01'
            band_indices = ge01_band_indices
        else:
            raise ValueError
        for pre_gan in ['pre', 'gan', 'bicubic']:
            if pre_gan == 'pre':
                predict_model = esrgan_generator_model
                predict_model.load_weights(pretrain_weights_path)
            elif pre_gan == 'gan':
                predict_model = esrgan_generator_model
                predict_model.load_weights(gan_weights_path)
            else:
                predict_model = bicubic_model
            sr_img = esrgan_predict(
                model=predict_model,
                ms_img_path=ms_tile_path,
                result_dir=results_img_dir,
                sensor=sensor,
                band_indices=band_indices,
                mean_correction=mean_correction,
                pre_or_gan=pre_gan,
                sr_factor=sr_factor,
                copy_pan_img=True,
                pan_img_path=None,
                output_dtype=output_dtype,
                geotiff_or_png=geotiff_or_png)
        count += 1
        print(count, 'Saved ms, pre, gan and pan image for patch', results_img_dir, ms_tile_path.name)

In [ ]:
ms_tiles_paths = list_all_ms_tiles('data/toulon-laspezia-tiles/e07/test/')
print(len(ms_tiles_paths))

esrgan_predict_multiple(
    ms_tiles_paths=ms_tiles_paths, 
    results_dir='results/thesis-appendix/', 
    esrgan_generator_model=pretrain_model,
    bicubic_model=bicubic,
    pretrain_weights_path=PRETRAIN_WEIGHTS_PATH, 
    gan_weights_path=GAN_WEIGHTS_PATH,
    wv02_band_indices=MS_BANDS_WV02_IDXS,
    ge01_band_indices=MS_BANDS_GE01_IDXS,
    mean_correction=train_tiles_mean,
    sr_factor=SR_FACTOR, 
    geotiff_or_png='png')

In [ ]:

    
IMG = 'GE01_Toulon 2014_10_16_011651185010_0'
#IMG = 'GE01_La_Spezia_2009_09_25_011651186010_0'
#IMG = 'GE01_La_Spezia_2019_01_03_011651196010_0'
#IMG = 'GE01_La_Spezia_2013_07_23_011651202010_0'
IMG = 'GE01_Toulon 2010_06_08_011651191010_0'
MS_IMG_PATH = 'data/toulon-laspezia-tiles/e07/test/' + IMG + '/ms/00227.tif'

RESULTS_DIR = 'results/thesis-appendix/' + IMG
SENSOR = 'GE01'
predict_model = pretrain_model
for pre_gan in ['pre', 'gan', 'bicubic']:
    if SENSOR == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    elif SENSOR == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    if pre_gan == 'pre':
        predict_model = pretrain_model
        predict_model.load_weights(PRETRAIN_WEIGHTS_PATH)
    elif pre_gan == 'gan':
        predict_model = pretrain_model
        predict_model.load_weights(GAN_WEIGHTS_PATH)
    else:
        predict_model = bicubic
    sr_img = esrgan_predict(model=predict_model, 
                            ms_img_path=MS_IMG_PATH, 
                            result_dir=RESULTS_DIR, 
                            sensor=SENSOR, 
                            band_indices=band_indices, 
                            mean_correction=train_tiles_mean, 
                            pre_or_gan=pre_gan,
                            sr_factor=SR_FACTOR, 
                            copy_pan_img=True,
                            pan_img_path=None, 
                            output_dtype='uint16', 
                            geotiff_or_png='png')